In [22]:
# Imports
import numpy as np

In [88]:
# Load file
data = np.genfromtxt("./scenario1/scenario_1.txt", delimiter=",")

## CoM

In [89]:
# CoM error between prediction and real
real_com = data[1:-1, :2]
predicted_com = data[1:-1, 2:4]

In [90]:
com_diff = np.abs(real_com - predicted_com)

In [91]:
print("Highest error between real and predicted com: ", np.max(com_diff[:, 0]), np.max(com_diff[:, 1]))
print("Lowest error between real and predicted com: ", np.min(com_diff[:, 0]), np.min(com_diff[:, 1]))
print("Average error between real and predicted com: ", np.mean(com_diff[:, 0]), np.mean(com_diff[:, 1]))

Highest error between real and predicted com:  0.06920000000000037 0.04952898
Lowest error between real and predicted com:  0.0010199999999995768 1.0799999999998311e-05
Average error between real and predicted com:  0.030900361702127715 0.014446455080851064


## Footsteps

In [92]:
# Feet error between predictions and real
real_footsteps = data[1:-1, 4:12]
predicted_footsteps = data[1:-1, 12:20]
print(real_footsteps.shape)
print(predicted_footsteps.shape)

(94, 8)
(94, 8)


In [93]:
# Swinging booleans
fr_rl_swinging = np.where(real_footsteps[:, 2] > real_footsteps[:, 0])[0]
fl_rr_swinging = np.where(real_footsteps[:, 0] > real_footsteps[:, 2])[0]
print(fr_rl_swinging.shape)
print(fl_rr_swinging.shape)

(47,)
(47,)


In [94]:
footsteps_diff = np.abs(real_footsteps - predicted_footsteps)

In [95]:
footsteps_diff[fr_rl_swinging, 2]

array([0.00022 , 0.001974, 0.00537 , 0.01276 , 0.01074 , 0.00577 ,
       0.00632 , 0.01318 , 0.01477 , 0.01487 , 0.02421 , 0.01021 ,
       0.00987 , 0.02323 , 0.0133  , 0.01212 , 0.02616 , 0.00969 ,
       0.01367 , 0.01442 , 0.01303 , 0.01144 , 0.01478 , 0.01075 ,
       0.03698 , 0.02412 , 0.01763 , 0.04116 , 0.00297 , 0.00447 ,
       0.00287 , 0.02027 , 0.00232 , 0.02975 , 0.02967 , 0.00642 ,
       0.02237 , 0.01366 , 0.00356 , 0.00912 , 0.00467 , 0.00662 ,
       0.00829 , 0.01028 , 0.00722 , 0.01131 , 0.01015 ])

In [96]:
print("Highest error for FL foot: ", np.max(footsteps_diff[fl_rr_swinging, 0]), np.max(footsteps_diff[fl_rr_swinging, 0]))
print("Lowest error for FL foot: ", np.min(footsteps_diff[fl_rr_swinging, 0]), np.min(footsteps_diff[fl_rr_swinging, 0]))
print("Average error for FL foot: ", np.mean(footsteps_diff[fl_rr_swinging, 0]), np.mean(footsteps_diff[fl_rr_swinging, 0]))

Highest error for FL foot:  0.020319999999999894 0.020319999999999894
Lowest error for FL foot:  0.00013000000000040757 0.00013000000000040757
Average error for FL foot:  0.004445574468085115 0.004445574468085115


In [97]:
print("Highest error for FR foot: ", np.max(footsteps_diff[fr_rl_swinging, 2]), np.max(footsteps_diff[fr_rl_swinging, 3]))
print("Lowest error for FR foot: ", np.min(footsteps_diff[fr_rl_swinging, 2]), np.min(footsteps_diff[fr_rl_swinging, 3]))
print("Average error for FR foot: ", np.mean(footsteps_diff[fr_rl_swinging, 2]), np.mean(footsteps_diff[fr_rl_swinging, 3]))

Highest error for FR foot:  0.04116000000000053 0.04800539999999999
Lowest error for FR foot:  0.00021999999999999797 0.00038299999999999446
Average error for FR foot:  0.013164553191489431 0.013901876595744678


In [98]:
print("Highest error for RL foot: ", np.max(footsteps_diff[fr_rl_swinging, 4]), np.max(footsteps_diff[fr_rl_swinging, 5]))
print("Lowest error for RL foot: ", np.min(footsteps_diff[fr_rl_swinging, 4]), np.min(footsteps_diff[fr_rl_swinging, 5]))
print("Average error for RL foot: ", np.mean(footsteps_diff[fr_rl_swinging, 4]), np.mean(footsteps_diff[fr_rl_swinging, 5]))

Highest error for RL foot:  0.029269999999999463 0.029488999999999987
Lowest error for RL foot:  0.0003389999999999782 0.0005469999999999919
Average error for RL foot:  0.010933323404255353 0.011010851063829787


In [99]:
print("Highest error for RR foot: ", np.max(footsteps_diff[fl_rr_swinging, 6]), np.max(footsteps_diff[fl_rr_swinging, 7]))
print("Lowest error for RR foot: ", np.min(footsteps_diff[fl_rr_swinging, 6]), np.min(footsteps_diff[fl_rr_swinging, 7]))
print("Average error for RR foot: ", np.mean(footsteps_diff[fl_rr_swinging, 6]), np.mean(footsteps_diff[fl_rr_swinging, 7]))

Highest error for RR foot:  0.014255000000000018 0.032999200000000006
Lowest error for RR foot:  0.0005099999999997884 0.00131669999999999
Average error for RR foot:  0.0037415702127659304 0.012900404255319145


## Edges

In [100]:
# X coordinates of the stairs climbing up
edges = [1 + 0.21428571428 * x for x in range(14)] + [4.2142857142 + 0.21428571428 * x for x in range(14)]

In [101]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [102]:
# Compute minimum distance from any obstacle
min_edge_distances = []
for real_footstep in real_footsteps:
    temp = []
    for idx in range(len(real_footstep)):
        if idx in [0, 2, 4, 6]:
            temp.append(find_nearest(edges, real_footstep[idx]))
    min_edge_distances.append(temp)

min_edge_distances = np.array(min_edge_distances)

In [103]:
real_edge_distances = np.abs(real_footsteps[:, ::2] - min_edge_distances)
predicted_edge_distances = np.abs(predicted_footsteps[:, ::2] - min_edge_distances)

In [104]:
print("Lowest distance for FL foot: ", np.min(real_edge_distances[fl_rr_swinging, 0]), np.min(predicted_edge_distances[fl_rr_swinging, 0]))
print("Number of contacts with less than 1cm distance: ", len(np.where(real_edge_distances[fl_rr_swinging, 0] < 0.01)[0]))
print("Percentage of edge contacts: ", len(np.where(real_edge_distances[fl_rr_swinging, 0] < 0.01)[0]) / len(real_edge_distances[fl_rr_swinging]))

Lowest distance for FL foot:  9.999996000020772e-05 0.0008042858000001374
Number of contacts with less than 1cm distance:  4
Percentage of edge contacts:  0.0851063829787234


In [105]:
print("Lowest distance for FR foot: ", np.min(real_edge_distances[fr_rl_swinging, 1]), np.min(predicted_edge_distances[fr_rl_swinging, 1]))
print("Number of contacts with less than 1cm distance: ", len(np.where(real_edge_distances[fr_rl_swinging, 1] < 0.01)[0]))
print("Percentage of edge contacts: ", len(np.where(real_edge_distances[fr_rl_swinging, 1] < 0.01)[0]) / len(real_edge_distances[fr_rl_swinging]))

Lowest distance for FR foot:  0.0017485714400000685 0.00027285708000013287
Number of contacts with less than 1cm distance:  2
Percentage of edge contacts:  0.0425531914893617


In [106]:
print("Lowest distance for RL foot: ", np.min(real_edge_distances[fr_rl_swinging, 2]), np.min(predicted_edge_distances[fr_rl_swinging, 2]))
print("Number of contacts with less than 1cm distance: ", len(np.where(real_edge_distances[fr_rl_swinging, 2] < 0.01)[0]))
print("Percentage of edge contacts: ", len(np.where(real_edge_distances[fr_rl_swinging, 2] < 0.01)[0]) / len(real_edge_distances[fr_rl_swinging]))

Lowest distance for RL foot:  0.0036257144400000385 0.01648428555999981
Number of contacts with less than 1cm distance:  3
Percentage of edge contacts:  0.06382978723404255


In [107]:
print("Lowest distance for RR foot: ", np.min(real_edge_distances[fl_rr_swinging, 3]), np.min(predicted_edge_distances[fl_rr_swinging, 3]))
print("Number of contacts with less than 1cm distance: ", len(np.where(real_edge_distances[fl_rr_swinging, 3] < 0.01)[0]))
print("Percentage of edge contacts: ", len(np.where(real_edge_distances[fl_rr_swinging, 3] < 0.01)[0]) / len(real_edge_distances[fl_rr_swinging]))

Lowest distance for RR foot:  0.00040857132000038376 0.0005685712800005405
Number of contacts with less than 1cm distance:  4
Percentage of edge contacts:  0.0851063829787234


## Velocities

In [108]:
print("Velocities commanded: ", set(data[:, -1]))

Velocities commanded:  {0.9, 0.7, 0.6, 0.8, 0.4}
